In [1]:
# 라이브러리 import 
import pandas as pd
import numpy as np
import scipy.stats as stats

In [2]:
# pandas 라이브러리를 활용한 csv 파일 읽기 
df = pd.read_csv("product_details.csv") # product_details.csv
df2 = pd.read_csv("customer_details.csv") # customer_details.csv
df3 = pd.read_csv("E-commerece sales data 2024.csv") # E-commerece sales data 2024.csv

In [3]:
# 결측값 처리
df3 = df3.fillna(0)
df3.drop(['Unnamed: 4','product id'], axis=1, inplace=True)
df3['Time stamp'] = pd.to_datetime(df3['Time stamp'],infer_datetime_format=True)
df3['Time stamp'] = df3['Time stamp'].dt.strftime('%Y-%m-%d')
df3_prac1 = df3.copy()
df3_prac2 = df3.copy()

In [4]:
df3_prac1

,user id,Interaction type,Time stamp
0,1.0,purchase,2023-10-10
1,2.0,view,2023-11-10
2,3.0,like,2023-12-10
3,4.0,view,2023-10-13
4,5.0,like,2023-10-14
...,...,...,...
3289,0.0,0,1970-01-01
3290,0.0,0,1970-01-01
3291,0.0,0,1970-01-01
3292,0.0,0,1970-01-01


In [5]:
# shift method
df3_prac1['T_SHIFT1'] = df3_prac1['Time stamp'].shift(1) 
df3_prac1['T_SHIFT-1'] = df3_prac1['Time stamp'].shift(-1) 
df3_prac1

,user id,Interaction type,Time stamp,T_SHIFT1,T_SHIFT-1
0,1.0,purchase,2023-10-10,None,2023-11-10
1,2.0,view,2023-11-10,2023-10-10,2023-12-10
2,3.0,like,2023-12-10,2023-11-10,2023-10-13
3,4.0,view,2023-10-13,2023-12-10,2023-10-14
4,5.0,like,2023-10-14,2023-10-13,2023-10-15
...,...,...,...,...,...
3289,0.0,0,1970-01-01,1970-01-01,1970-01-01
3290,0.0,0,1970-01-01,1970-01-01,1970-01-01
3291,0.0,0,1970-01-01,1970-01-01,1970-01-01
3292,0.0,0,1970-01-01,1970-01-01,1970-01-01


In [6]:
# rolling 메서드 활용을 위한 merge
merged_df = pd.merge(df3_prac2, df2, left_on='user id', right_on='Customer ID',how='inner')

In [7]:
# group by 응용연산
merged_df= merged_df[['Customer ID','Purchase Amount (USD)','Time stamp']]
merged_df = merged_df.groupby(['Time stamp']).agg({"Customer ID":"count","Purchase Amount (USD)":"sum"}).reset_index()
# expanding 실습때 사용할 데이터셋 미리 복제
merged_df2 = merged_df.copy()

In [8]:
# rolling 매서드를 활용한 purchase amount 3일 이동평균 구하기 
# 이동평균이란? 
# n번째 데이터의 단순이동평균 = n번째 데이터를 포함한 이전날짜 m개의 데이터의 산술평균

# 윈도우크기 설정
window_size = 3

# 3일 이동평균을 구하고 원본데이터의 새로운 컬럼으로 받기
# 0,1 번은 이전날짜가 없어 NaN(not a number) 로 반환됩니다. 
merged_df['Moving Average']=merged_df['Purchase Amount (USD)'].rolling(window=window_size).mean().round(1)
merged_df

,Time stamp,Customer ID,Purchase Amount (USD),Moving Average
0,2022-12-28,18,1187,NaN
1,2022-12-29,18,985,NaN
2,2022-12-30,18,995,1055.7
3,2022-12-31,18,1018,999.3
4,2023-01-01,36,2481,1498.0
...,...,...,...,...
360,2023-12-24,18,1091,1110.0
361,2023-12-25,18,1181,1115.3
362,2023-12-26,18,947,1073.0
363,2023-12-27,18,1159,1095.7


In [9]:
# rolling 메서드의 center 옵션 확인하기 
# 3일 이동평균을 구하고 원본데이터의 새로운 컬럼으로 받기
# cetner = True 로 인해 중간행을 기준으로 계산됩니다. 
# closed = left 로 인해 왼쪽 값을 포함하여 계산합니다. 기본값을 right 입니다.
merged_df['Moving Average2']=merged_df['Purchase Amount (USD)'].rolling(window=window_size, center=True).mean().round(1)
merged_df

,Time stamp,Customer ID,Purchase Amount (USD),Moving Average,Moving Average2
0,2022-12-28,18,1187,NaN,NaN
1,2022-12-29,18,985,NaN,1055.7
2,2022-12-30,18,995,1055.7,999.3
3,2022-12-31,18,1018,999.3,1498.0
4,2023-01-01,36,2481,1498.0,1266.7
...,...,...,...,...,...
360,2023-12-24,18,1091,1110.0,1115.3
361,2023-12-25,18,1181,1115.3,1073.0
362,2023-12-26,18,947,1073.0,1095.7
363,2023-12-27,18,1159,1095.7,1070.0


In [10]:
# rolling 매서드를 활용한 purchase amount 3일 누적합 구하기 
# 누적합계란?
# n번째 데이터의 누적합 = n번째 데이터를 포함한 이전날짜 m개의 데이터의 누적합계

# 윈도우크기 설정
window_size = 3

# 3일 누적합을 구하고 원본데이터의 새로운 컬럼으로 받기
# 0,1 번은 이전날짜가 없어 NaN(not a number) 로 반환됩니다. 
merged_df['prefix_sum']=merged_df['Purchase Amount (USD)'].rolling(window=window_size).sum().round(1)
merged_df

,Time stamp,Customer ID,Purchase Amount (USD),Moving Average,Moving Average2,prefix_sum
0,2022-12-28,18,1187,NaN,NaN,NaN
1,2022-12-29,18,985,NaN,1055.7,NaN
2,2022-12-30,18,995,1055.7,999.3,3167.0
3,2022-12-31,18,1018,999.3,1498.0,2998.0
4,2023-01-01,36,2481,1498.0,1266.7,4494.0
...,...,...,...,...,...,...
360,2023-12-24,18,1091,1110.0,1115.3,3330.0
361,2023-12-25,18,1181,1115.3,1073.0,3346.0
362,2023-12-26,18,947,1073.0,1095.7,3219.0
363,2023-12-27,18,1159,1095.7,1070.0,3287.0


In [11]:
# closed 옵션 비교하기 
a = pd.DataFrame({'values': [10, 20, 30, 40, 50]})
# 현재 위치를 포함하여 데이터가 3개인 순간부터 계산 
aa = a.rolling(window=3).sum()

# 현재 위치를 포함하지 않고 데이터가 3개인 순간부터 계산
b = a.rolling(window=3, closed='left').sum() 

# right 와 동일하게 동작. 현재위치와 양쪽 경계를 포함하여 데이터가 3개인 순간부터 계산
c = a.rolling(window=3, closed='both').sum() 

# 양쪽 경계와 현재위치를 사용하지않고 윈도우 크기 3을 만족할 경우에만 출력 
# window 가 3인 상태에서, 양쪽 경계값을 제외하면 1개의 데이터가 남으므로 모든 값이 NaN으로 출력. 
# 무의미
d = a.rolling(window=3, closed='neither').sum() 

#결과표 한번에 보여주기 
e = pd.concat([a,aa, b,c,d], axis=1)
e.columns = ['원본','right','left','both','neither']

e

,원본,right,left,both,neither
0,10,NaN,NaN,NaN,NaN
1,20,NaN,NaN,NaN,NaN
2,30,60.0,NaN,60.0,NaN
3,40,90.0,60.0,100.0,NaN
4,50,120.0,90.0,140.0,NaN


In [12]:
# Expanding을 이용한 누적 평균 및 누적 합계 계산
merged_df["Cumulative Mean Purchase"] = merged_df["Purchase Amount (USD)"].expanding().mean()
merged_df["Cumulative Sum Purchase"] = merged_df["Purchase Amount (USD)"].expanding().sum()
merged_df

,Time stamp,Customer ID,Purchase Amount (USD),Moving Average,Moving Average2,prefix_sum,Cumulative Mean Purchase,Cumulative Sum Purchase
0,2022-12-28,18,1187,NaN,NaN,NaN,1187.000000,1187.0
1,2022-12-29,18,985,NaN,1055.7,NaN,1086.000000,2172.0
2,2022-12-30,18,995,1055.7,999.3,3167.0,1055.666667,3167.0
3,2022-12-31,18,1018,999.3,1498.0,2998.0,1046.250000,4185.0
4,2023-01-01,36,2481,1498.0,1266.7,4494.0,1333.200000,6666.0
...,...,...,...,...,...,...,...,...
360,2023-12-24,18,1091,1110.0,1115.3,3330.0,483.404432,174509.0
361,2023-12-25,18,1181,1115.3,1073.0,3346.0,485.331492,175690.0
362,2023-12-26,18,947,1073.0,1095.7,3219.0,486.603306,176637.0
363,2023-12-27,18,1159,1095.7,1070.0,3287.0,488.450549,177796.0


In [13]:
# 상관관계 실습 시작 
df = pd.read_csv('articles_hm.csv')
df2 = pd.read_csv('customer_hm.csv')
df3 = pd.read_csv('transactions_hm.csv')

In [14]:
# 상관관계 실습을 위한 데이터 전처리
mdf = pd.merge(df2, df3, how='inner', on='customer_id')
mdf2=pd.merge(mdf, df, how='inner', on='article_id')

In [15]:
mdf2.head().T

,0,1,2,3,4
customer_id,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,00159bfc26c2cf788c09789f006fa698b24d0f1dbd8309...,00c0df20797235cc21ce557cc7d9bd59df668e77a67035...,00ce2da7f87dbcc49a3c988ae138d54bc058307290fb44...,011342558430acfbbea2b19a0f822558ef0066ccf19b0d...
FN,0,0,1,0,0
Active,0,0,1,0,0
club_member_status,ACTIVE,ACTIVE,ACTIVE,ACTIVE,ACTIVE
fashion_news_frequency,NONE,NONE,Regularly,NONE,NONE
age,49,53,23,52,20
t_dat,2019-05-25,2019-02-19,2019-08-12,2019-06-04,2019-06-19
article_id,568601006,568601006,568601006,568601006,568601006
price,0.050831,0.050831,0.045746,0.050831,0.045746
sales_channel_id,2,1,2,2,1


In [16]:
# 범주형 변수의 범위 확인하기 1
mdf2['club_member_status'].unique()

array(['ACTIVE', 'PRE-CREATE', 'LEFT CLUB'], dtype=object)

In [17]:
# 범주형 변수의 범위 확인하기 2
mdf2['fashion_news_frequency'].unique()

array(['NONE', 'Regularly', 'Monthly'], dtype=object)

In [18]:
# 범주형 변수의 범위 확인하기 3
mdf2['sales_channel_id'].unique()

array([2, 1], dtype=int64)

In [19]:
# Point-Biserial Correlation
# 연속형 변수와 이분형 범주형 변수에 대한 상관관계 구하기 
r, p_value = stats.pointbiserialr(mdf['sales_channel_id'], mdf['price'])
r, p_value

# r = 0.17 로 상관관계가 높지 않음
# p-value 가 0.05 보다 작으므로, 이는 통계적으로 신뢰를 가지고 판단할 수 있음 

(0.17081898530591685, 0.0)

In [20]:
# ANOVA
# 연속형 변수의 평균이 범주형 변수에 따라 차이가 있는지 분석하는 방법
# 범주형 변수별 그룹 분할
# 카테고리를 club_member_status 의 unique 값들로 지정하고, 이를 모든 행에 적용(for문), 결과적으로 필터링 된 데이터셋 중에서
# price 컬럼만 가져오는 코드 
groups = [mdf2['price'][mdf2['club_member_status'] == category] for category in mdf2['club_member_status'].unique()]

# ANOVA 검정 수행
# F <1 : 무의미
# 1 <= F < 3 : 거의 무의미 
# 3 <= F <10 : 경우에 따라 유의미
# 10 <= F < 50 : 유의미(강한차이)
# F > 50 : 거의 확실한 유의미(아주 강한차이)
# F >= 100 : 확실한 유의미(매우 강한차이)
# F 값이 큰것만 확인하는 것이 아니라, 이를 통계적으로 설명 가능한 지 P-value 도 함께 봐야 합니다. (즉, 0.05 미만인지 봐야합니다.)
# 이 말은 곧, F 값이 크게 나왔지만, P-value 가 0.05 보다 크다면, 통계적으로 설명할 수 없다는 것을 의미합니다. 
# 클럽 멤버상태와 결제 금액은 매우 강한 상관관계가 있습니다. 

anova_result = stats.f_oneway(*groups)
print(f"ANOVA 검정 결과: F={anova_result.statistic:.4f}, p-value={anova_result.pvalue:.4f}")

ANOVA 검정 결과: F=150.2724, p-value=0.0000


In [21]:
# Cramer' V
# 범주형 변수가 3 개 이상인 경우 크래머 V계수(Cramer's V) 사용
from scipy.stats.contingency import association
from sklearn import preprocessing
from scipy.stats import chi2_contingency

In [22]:
# step 01. label encoding 
base_df = mdf2[['club_member_status','fashion_news_frequency']]
base_df = base_df.dropna()

label = preprocessing.LabelEncoder()
data_encoded = pd.DataFrame() 

for i in base_df.columns :
      data_encoded[i]=label.fit_transform(base_df[i])
data_encoded.head()

,club_member_status,fashion_news_frequency
0,0,1
1,0,1
2,0,2
3,0,1
4,0,1


In [26]:
data_encoded.groupby(['club_member_status']).count()

,fashion_news_frequency
club_member_status,
0,796063
1,260
2,16608


In [23]:
# step 02-03.함수 정의 및 혼동행렬 생성(define a function and make a confusion matrix)

def cramers_V(var1,var2) :
    crosstab =np.array(pd.crosstab(var1,var2, rownames=None, colnames=None)) # Cross table building
    stat = chi2_contingency(crosstab)[0] # Keeping of the test statistic of the Chi2 test
    obs = np.sum(crosstab) # Number of observations
    phi2 = stat / obs
    r, k = crosstab.shape
    phi2corr = max(0, phi2 - (((k-1)*(r-1))/(obs - 1)))
    rcorr = r - ((r-1)**2)/(obs-1)
    kcorr = k - ((k-1)**2)/(obs-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

rows= []

for var1 in data_encoded:
    col = []
    for var2 in data_encoded :
        cramers =cramers_V(data_encoded[var1], data_encoded[var2]) # Cramer's V test
        col.append(round(cramers,2)) # Keeping of the rounded value of the Cramer's V  
    rows.append(col)
cramers_results = np.array(rows)
data_encoded_df = pd.DataFrame(cramers_results, columns = data_encoded.columns, index =data_encoded.columns)

data_encoded_df

,club_member_status,fashion_news_frequency
club_member_status,1.00,0.07
fashion_news_frequency,0.07,1.00
